# STIX Patterns

Python-stix2 supports STIX2 patterning insofar that patterns may be used for the pattern property of Indicators, identical to the STIX2 specification. Python-stix2 does not evaluate patterns against STIX2 content, for that functionality see [cti-pattern-matcher].(https://github.com/oasis-open/cti-pattern-matcher)

##  Intro

Python-stix2 patterns are built compositely from the bottom up, creating components at sublevels before those at higher levels.

## Examples

### Comparison Expressions

In [5]:
from stix2 import DomainName, File, IPv4Address
from stix2 import (ObjectPath, EqualityComparisonExpression, GreaterThanComparisonExpression,
                   IsSubsetComparisonExpression, FloatConstant, StringConstant)

# ---- Equality Comparison expressions
print("Equality Comparison Expressions:\n")

lhs = ObjectPath("domain-name", ["value"])
ece_1 = EqualityComparisonExpression(lhs, "site.of.interest.zaz")
print("\t{}\n".format(ece_1.to_pattern()))

lhs = ObjectPath("file", ["parent_directory_ref","path"])
ece_2 = EqualityComparisonExpression(lhs, "C:\\Windows\\System32")
print("\t{}\n".format(ece_2.to_pattern()))

# Greater-than Comparison expressions
print("\nGreater-than Comparison Expression:\n")

lhs = ObjectPath("file", ["extensions", "windows-pebinary-ext", "sections[*]", "entropy"])
gte = GreaterThanComparisonExpression(lhs, FloatConstant("7.0"))
print("\t{}\n".format(gte.to_pattern()))

# IsSubset Comparison expressions
print("\nIs-Subset Comparison Expression:\n")

lhs = ObjectPath("network-traffic", ["dst_ref", "value"])
iss = IsSubsetComparisonExpression(lhs, StringConstant("2001:0db8:dead:beef:0000:0000:0000:0000/64"))
print("\t{}\n".format(iss.to_pattern()))



Equality Comparison Expressions:

	[domain-name:value = 'site.of.interest.zaz']

	[file:parent_directory_ref.path = 'C:\\Windows\\System32']


Greater-than Comparison Expression:

	[file:extensions.windows-pebinary-ext.sections[*].entropy > 7.0]


Is-Subset Comparison Expression:

	[network-traffic:dst_ref.value ISSUBSET '2001:0db8:dead:beef:0000:0000:0000:0000/64']



### Observation Expressions

In [2]:
from stix2 import (IntegerConstant, HashConstant, ObjectPath,
                   EqualityComparisonExpression, AndBooleanExpression,
                   OrBooleanExpression, ParentheticalExpression,
                   AndObservationExpression, OrObservationExpression,
                   FollowedByObservationExpression)

# ---- Observation expressions
print("Observation Expressions:\n")

# AND boolean
ece3 = EqualityComparisonExpression(ObjectPath("email-message", ["sender_ref", "value"]), "stark@example.com")
ece4 = EqualityComparisonExpression(ObjectPath("email-message", ["subject"]), "Conference Info")
abe = AndBooleanExpression([ece3, ece4])
print("(AND)\n{}\n".format(abe.to_pattern()))

# OR boolean
ece5 = EqualityComparisonExpression(ObjectPath("url", ["value"]), "http://example.com/foo")
ece6 = EqualityComparisonExpression(ObjectPath("url", ["value"]), "http://example.com/bar")
obe = OrBooleanExpression([ece5, ece6])
print("(OR)\n{}\n".format(obe.to_pattern()))

# ( OR ) AND   boolean
ece7 = EqualityComparisonExpression(ObjectPath("file", ["name"]), "pdf.exe")
ece8 = EqualityComparisonExpression(ObjectPath("file", ["size"]), IntegerConstant("371712"))
ece9 = EqualityComparisonExpression(ObjectPath("file", ["created"]), "2014-01-13T07:03:17Z")
obe1 = OrBooleanExpression([ece7, ece8])
pobe = ParentheticalExpression(obe1)
abe1 = AndBooleanExpression([pobe, ece9])
print("(OR,AND)\n{}\n".format(abe1.to_pattern()))

# ( AND ) OR ( OR )  observation
ece20 = EqualityComparisonExpression(ObjectPath("file", ["name"]), "foo.dll")
ece21 = EqualityComparisonExpression(ObjectPath("win-registry-key", ["key"]), "HKEY_LOCAL_MACHINE\\foo\\bar")
ece22 = EqualityComparisonExpression(ObjectPath("process", ["name"]), "fooproc")
ece23 = EqualityComparisonExpression(ObjectPath("process", ["name"]), "procfoo")
# NOTE: we need to use AND/OR observation expression instead of just boolean 
# expressions as the operands are not on the same object-type
aoe = ParentheticalExpression(AndObservationExpression([ece20, ece21]))
obe2 = OrBooleanExpression([ece22, ece23])
ooe = OrObservationExpression([aoe, obe2])
print("(AND,OR,OR)\n{}\n".format(ooe.to_pattern()))

# FOLLOWED-BY
ece10 = EqualityComparisonExpression(ObjectPath("file", ["hashes", "MD5"]), HashConstant("79054025255fb1a26e4bc422aef54eb4", "MD5"))
ece11 = EqualityComparisonExpression(ObjectPath("win-registry-key", ["key"]), "HKEY_LOCAL_MACHINE\\foo\\bar")
fbe = FollowedByObservationExpression([ece10, ece11])
print("(FollowedBy)\n{}\n".format(fbe.to_pattern()))

Observation Expressions:

(AND)
[email-message:sender_ref.value = 'stark@example.com' AND email-message:subject = 'Conference Info']

(OR)
[url:value = 'http://example.com/foo' OR url:value = 'http://example.com/bar']

(OR,AND)
[(file:name = 'pdf.exe' OR file:size = 371712) AND file:created = 2014-01-13 07:03:17+00:00]

(AND,OR,OR)
([file:name = 'foo.dll'] AND [win-registry-key:key = 'HKEY_LOCAL_MACHINE\\foo\\bar']) OR [process:name = 'fooproc' OR process:name = 'procfoo']

(FollowedBy)
[file:hashes.MD5 = '79054025255fb1a26e4bc422aef54eb4'] FOLLOWEDBY [win-registry-key:key = 'HKEY_LOCAL_MACHINE\\foo\\bar']



__See [note](#BooleanExpressions-vs-CompoundObservationExpressions) on when to use BooleanExpressions vs CompoundObservationExpressions__

### Qualified Observation Expressions

In [3]:
from stix2 import (TimestampConstant, HashConstant, ObjectPath, EqualityComparisonExpression,
                   AndBooleanExpression, WithinQualifier, RepeatQualifier, StartStopQualifier,
                   QualifiedObservationExpression, FollowedByObservationExpression)

# Qualified Observation Expressions
print("Qualified Observation Expressions:\n")

# WITHIN
ece10 = EqualityComparisonExpression(ObjectPath("file", ["hashes", "MD5"]), HashConstant("79054025255fb1a26e4bc422aef54eb4", "MD5"))
ece11 = EqualityComparisonExpression(ObjectPath("win-registry-key", ["key"]), "HKEY_LOCAL_MACHINE\\foo\\bar")
fbe = FollowedByObservationExpression([ece10, ece11])
qoe = QualifiedObservationExpression(fbe, WithinQualifier(300))
print("(WITHIN)\n{}\n".format(qoe.to_pattern()))

# REPEATS, WITHIN
ece12 = EqualityComparisonExpression(ObjectPath("network-traffic", ["dst_ref", "type"]), "domain-name")
ece13 = EqualityComparisonExpression(ObjectPath("network-traffic", ["dst_ref", "value"]), "example.com")
abe2 = AndBooleanExpression([ece12, ece13])
qoe1 = QualifiedObservationExpression(QualifiedObservationExpression(abe2, RepeatQualifier(5)), WithinQualifier(180))
print("(REPEAT, WITHIN)\n{}\n".format(qoe1.to_pattern()))

# START, STOP
ece14 = EqualityComparisonExpression(ObjectPath("file", ["name"]), "foo.dll")
ssq = StartStopQualifier(TimestampConstant('2016-06-01T00:00:00Z'), TimestampConstant('2016-07-01T00:00:00Z'))
qoe2 = QualifiedObservationExpression(ece14, ssq)
print("(START-STOP)\n{}\n".format(qoe2.to_pattern()))


Qualified Observation Expressions:

(WITHIN)
[file:hashes.MD5 = '79054025255fb1a26e4bc422aef54eb4'] FOLLOWEDBY [win-registry-key:key = 'HKEY_LOCAL_MACHINE\\foo\\bar'] WITHIN 300 SECONDS

(REPEAT, WITHIN)
[network-traffic:dst_ref.type = 'domain-name' AND network-traffic:dst_ref.value = 'example.com'] REPEATS 5 TIMES WITHIN 180 SECONDS

(START-STOP)
[file:name = 'foo.dll'] START t'2016-06-01T00:00:00Z' STOP t'2016-07-01T00:00:00Z'



## Attaching patterns to STIX2 Domain objects

As seen in the previous examples, the *__to_pattern()__* utility is used to display the pattern. However, the true purpose of this utility is to be able to seamlessly add a constructed pattern to an python-stix2 object.

The *__to_pattern()__* utility constructs a string version of the pattern that adheres to the STIX2 patterning language specification.

### Example

In [4]:
from stix2 import Indicator, EqualityComparisonExpression

ece14 = EqualityComparisonExpression(ObjectPath("file", ["name"]), "$$t00rzch$$.elf")
ind = Indicator(name="Cryptotorch", labels=["malware", "ransomware"], pattern="[{}]".format(ece14))
print(ind)

{
    "type": "indicator",
    "id": "indicator--9f2d8014-3c2f-4127-bc2c-a209913404a5",
    "created": "2018-08-27T18:57:00.762Z",
    "modified": "2018-08-27T18:57:00.762Z",
    "name": "Cryptotorch",
    "pattern": "[file:name = '$$t00rzch$$.elf']",
    "valid_from": "2018-08-27T18:57:00.762972Z",
    "labels": [
        "malware",
        "ransomware"
    ]
}


## BooleanExpressions vs CompoundObservationExpressions

Be careful to note the difference between these two very similar pattern components. 

__BooleanExpressions__
 - stix2.AndBooleanExpression
 - stix2.booleanExpression
 
  __Usage__: When the boolean sub-expressions refer to the same root object 

  __Example__:
    ```[domain-name:value = "www.5z8.info" AND domain-name:resolvess_to_refs[*].value = "'198.51.100.1/32'"]```
    
  __Rendering__: when pattern is rendered, brackets or parenthesis will encapsulate boolean expression
 
__CompoundObservationExpressions__
 - stix2.AndObservationExpression
 - stix2.OrObservationExpression
 
  __Usage__: When the boolean sub-expressions refer to different root objects

  __Example__:
    ```[file:name="foo.dll"] AND [process:name = "procfoo"]```
    
  __Rendering__: when pattern is rendered, brackets will encapsulate each boolean sub-expression
 
